In [13]:
# Do all imports and installs here - Done
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
import pandas as pd
import re
import configparser
import os

In [14]:
config = configparser.ConfigParser()
config.read('etl.cfg')

input_data_source = config.get('DIR','INPUT_DIR')
output_processed_data = config.get('DIR','OUTPUT_DIR')

i94immi_dataset = config.get('DATA','I94_IMMI')
worldtempe_dataset = config.get('DATA','WORLD_TEMPE')
citydemo_dataset = config.get('DATA','CITY_DEMOGRAPHIC')
airport_dataset = config.get('DATA','AIR_PORT')

In [15]:
# Create Spark session - Using for droduction only
spark = SparkSession.builder\
            .config("spark.jars.repositories", "https://repos.spark-packages.org/")\
            .config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11")\
            .enableHiveSupport()\
            .getOrCreate()

In [16]:
citydemo_dataset = './us-cities-demographics.csv'
citydemo_df = pd.read_csv(citydemo_dataset,sep=";")
citydemo_df.columns

Index(['City', 'State', 'Median Age', 'Male Population', 'Female Population',
       'Total Population', 'Number of Veterans', 'Foreign-born',
       'Average Household Size', 'State Code', 'Race', 'Count'],
      dtype='object')

In [17]:
citydemo_df.describe()

,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,Count
count,2891.000000,2.888000e+03,2.888000e+03,2.891000e+03,2878.000000,2.878000e+03,2875.000000,2.891000e+03
mean,35.494881,9.732843e+04,1.017696e+05,1.989668e+05,9367.832523,4.065360e+04,2.742543,4.896377e+04
std,4.401617,2.162999e+05,2.315646e+05,4.475559e+05,13211.219924,1.557491e+05,0.433291,1.443856e+05
min,22.900000,2.928100e+04,2.734800e+04,6.321500e+04,416.000000,8.610000e+02,2.000000,9.800000e+01
25%,32.800000,3.928900e+04,4.122700e+04,8.042900e+04,3739.000000,9.224000e+03,2.430000,3.435000e+03
50%,35.300000,5.234100e+04,5.380900e+04,1.067820e+05,5397.000000,1.882200e+04,2.650000,1.378000e+04
75%,38.000000,8.664175e+04,8.960400e+04,1.752320e+05,9368.000000,3.397175e+04,2.950000,5.444700e+04
max,70.500000,4.081698e+06,4.468707e+06,8.550405e+06,156961.000000,3.212500e+06,4.980000,3.835726e+06


In [18]:
pd.set_option('display.max_columns', 50)
citydemo_df.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [19]:
citydemo_df.shape

(2891, 12)

In [20]:
# Check unique
citydemo_df.nunique()

City                       567
State                       49
Median Age                 180
Male Population            593
Female Population          594
Total Population           594
Number of Veterans         577
Foreign-born               587
Average Household Size     161
State Code                  49
Race                         5
Count                     2785
dtype: int64

In [21]:
citydemo_df.isna().sum()

City                       0
State                      0
Median Age                 0
Male Population            3
Female Population          3
Total Population           0
Number of Veterans        13
Foreign-born              13
Average Household Size    16
State Code                 0
Race                       0
Count                      0
dtype: int64

Remove leading and trailing space characters at  and upcase values on column 'City' to merge later

In [22]:
citydemo_df.City = citydemo_df.City.str.strip().str.upper()
citydemo_df

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,SILVER SPRING,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,QUINCY,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,HOOVER,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,RANCHO CUCAMONGA,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,NEWARK,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402
...,...,...,...,...,...,...,...,...,...,...,...,...
2886,STOCKTON,California,32.5,150976.0,154674.0,305650,12822.0,79583.0,3.16,CA,American Indian and Alaska Native,19834
2887,SOUTHFIELD,Michigan,41.6,31369.0,41808.0,73177,4035.0,4011.0,2.27,MI,American Indian and Alaska Native,983
2888,INDIANAPOLIS,Indiana,34.1,410615.0,437808.0,848423,42186.0,72456.0,2.53,IN,White,553665
2889,SOMERVILLE,Massachusetts,31.0,41028.0,39306.0,80334,2103.0,22292.0,2.43,MA,American Indian and Alaska Native,374


Unique values of column 'Race' just be 5. We check duplicate of this column

In [23]:
race_unique = citydemo_df['Race'].unique()
race_unique

array(['Hispanic or Latino', 'White', 'Asian',
       'Black or African-American', 'American Indian and Alaska Native'],
      dtype=object)

Sampling of values 'American Indian and Alaska Native'

In [24]:
citydemo_df[citydemo_df['Race'] == 'American Indian and Alaska Native']

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
5,PEORIA,Illinois,33.1,56229.0,62432.0,118661,6634.0,7517.0,2.40,IL,American Indian and Alaska Native,1343
11,FOLSOM,California,40.9,41051.0,35317.0,76368,4187.0,13234.0,2.62,CA,American Indian and Alaska Native,998
14,WICHITA,Kansas,34.6,192354.0,197601.0,389955,23978.0,40270.0,2.56,KS,American Indian and Alaska Native,8791
17,LAREDO,Texas,28.8,124305.0,131484.0,255789,4921.0,68427.0,3.66,TX,American Indian and Alaska Native,1253
22,BOLINGBROOK,Illinois,33.7,36295.0,35801.0,72096,2951.0,15212.0,3.42,IL,American Indian and Alaska Native,323
...,...,...,...,...,...,...,...,...,...,...,...,...
2878,CHARLOTTE,North Carolina,34.3,396646.0,430475.0,827121,36046.0,128897.0,2.52,NC,American Indian and Alaska Native,8746
2882,DAVIS,California,26.3,33493.0,34163.0,67656,2176.0,13997.0,2.69,CA,American Indian and Alaska Native,779
2886,STOCKTON,California,32.5,150976.0,154674.0,305650,12822.0,79583.0,3.16,CA,American Indian and Alaska Native,19834
2887,SOUTHFIELD,Michigan,41.6,31369.0,41808.0,73177,4035.0,4011.0,2.27,MI,American Indian and Alaska Native,983


Check duplication of combination of key 'City' and 'Race'

In [25]:
citydemo_df[citydemo_df[['City','Race']].duplicated()].count()

City                      143
State                     143
Median Age                143
Male Population           143
Female Population         143
Total Population          143
Number of Veterans        143
Foreign-born              143
Average Household Size    143
State Code                143
Race                      143
Count                     143
dtype: int64

A lot duplicate

Try the combination of 'City', 'Race' and 'State'

In [26]:
citydemo_df[citydemo_df[['City','Race','State']].duplicated()].count()

City                      0
State                     0
Median Age                0
Male Population           0
Female Population         0
Total Population          0
Number of Veterans        0
Foreign-born              0
Average Household Size    0
State Code                0
Race                      0
Count                     0
dtype: int64

The key of this table will be combination of **['City','Race','State']**

Next, we convert column names to lowercase without space

In [27]:
def convert_column_names(df):
    cols = df.columns
    column_name_changed = []

    for col in cols:
        new_column = col.lstrip().rstrip().lower().replace (" ", "_").replace ("-", "_") #strip beginning spaces, makes lowercase, add underscpre
        column_name_changed.append(new_column)

    df.columns = column_name_changed

In [29]:
convert_column_names(citydemo_df)
citydemo_df.columns

Index(['city', 'state', 'median_age', 'male_population', 'female_population',
       'total_population', 'number_of_veterans', 'foreign_born',
       'average_household_size', 'state_code', 'race', 'count'],
      dtype='object')

Perform NaN inventory on columns **['male_population', 'female_population', 'number_of_veterans', 'foreign_born', 'average_household_size']**

In [30]:
def nan_percentage_calc(df):
    nan_demographics_df = pd.DataFrame(data=df.isnull().sum(), columns=['NaN'])
    nan_demographics_df.drop(nan_demographics_df[nan_demographics_df['NaN'] == 0].index, inplace = True)
    nan_demographics_df['% of NaN'] = (nan_demographics_df['NaN']/df.count())*100
    return nan_demographics_df

In [31]:
nan_percentage_calc(citydemo_df)

,NaN,% of NaN
male_population,3,0.103878
female_population,3,0.103878
number_of_veterans,13,0.451703
foreign_born,13,0.451703
average_household_size,16,0.556522


Remove records with missing value **['male_population', 'female_population', 'number_of_veterans', 'foreign_born', 'average_household_size']**

In [32]:
citydemo_df = citydemo_df.dropna(subset=["male_population","female_population","number_of_veterans","foreign_born","average_household_size"],how="any")

Verify duplicate again

In [34]:
citydemo_df.duplicated().sum()

0

There is no duplicate anymore :)

Let's do data statistical

In [35]:
citydemo_df.describe()

,median_age,male_population,female_population,total_population,number_of_veterans,foreign_born,average_household_size,count
count,2875.000000,2.875000e+03,2.875000e+03,2.875000e+03,2875.000000,2.875000e+03,2875.000000,2.875000e+03
mean,35.434678,9.744502e+04,1.018469e+05,1.992919e+05,9361.714435,4.069181e+04,2.742543,4.886379e+04
std,4.250501,2.167572e+05,2.320513e+05,4.487144e+05,13216.754474,1.558259e+05,0.433291,1.446315e+05
min,22.900000,2.928100e+04,2.734800e+04,6.321500e+04,416.000000,8.610000e+02,2.000000,9.800000e+01
25%,32.800000,3.931400e+04,4.122700e+04,8.043800e+04,3737.500000,9.224000e+03,2.430000,3.454000e+03
50%,35.300000,5.233600e+04,5.380900e+04,1.067820e+05,5397.000000,1.883000e+04,2.650000,1.378000e+04
75%,38.000000,8.668750e+04,8.958900e+04,1.753080e+05,9368.000000,3.400300e+04,2.950000,5.414650e+04
max,48.800000,4.081698e+06,4.468707e+06,8.550405e+06,156961.000000,3.212500e+06,4.980000,3.835726e+06


In [37]:
citydemo_df.head()

,city,state,median_age,male_population,female_population,total_population,number_of_veterans,foreign_born,average_household_size,state_code,race,count
0,SILVER SPRING,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,QUINCY,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,HOOVER,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,RANCHO CUCAMONGA,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,NEWARK,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


Saving the clean df to a new csv for staging

In [36]:
citydemo_df.to_csv('citydemo_df_clean.csv', index=False)